# Entrenamiento del modelo

### **1. Carga del dataset procesado**
Se utiliza el archivo limpio **`spotify_clean_modeling.csv`**, previamente generado en la etapa de preprocesamiento.

### **2. Separación de variables predictoras (X) y objetivo (y)**
- **X:** atributos musicales numéricos y categóricos.  
- **y:** variable binaria **`is_hit`**, que indica si una canción es considerada un “hit”.

### **3. Preparación del preprocesamiento**
- Identificación de columnas numéricas y categóricas.
- Aplicación de **OneHotEncoder** a las variables categóricas.
- Construcción de un **ColumnTransformer** para combinar:
  - Codificación One-Hot  
  - Paso directo de variables numéricas

### **4. Construcción del pipeline del modelo**
Se integra el preprocesador y el algoritmo dentro de un único **Pipeline**, garantizando un flujo reproducible y listo para producción.

### **5. División del conjunto de datos**
- Separación en entrenamiento y prueba mediante `train_test_split`.  
- Uso de `stratify=y` para mantener la proporción de clases.

### **6. Entrenamiento del modelo LightGBM**
- Ajuste del modelo utilizando `class_weight="balanced"` para corregir el fuerte desbalance de clases.  
- Entrenamiento del pipeline completo sobre el conjunto de entrenamiento.

### **7. Evaluación inicial del modelo**
Métricas calculadas sobre el conjunto de prueba:
- **Accuracy**  
- **F1-score** (clave por el desbalance)  
- **ROC-AUC**

### **8. Optimización del umbral de decisión (threshold tuning)**
- Evaluación del **F1-score** para múltiples umbrales.  
- Selección del threshold que maximiza la detección correcta de canciones exitosas.

### **9. Evaluación final con threshold optimizado**
- Comparación de métricas con el nuevo umbral seleccionado.  
- Confirmación de mejoras en la detección de la clase positiva (**hits**).

### **10. Guardado del modelo entrenado**
- Serialización del pipeline completo mediante **`joblib`**.  
- Generación del archivo **`lightgbm_hit_classifier.joblib`** para su uso futuro en:
  - Módulo de inferencia  
  - API `/songs/predict_hit`

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from lightgbm import LGBMClassifier
import os
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn import set_config


In [74]:

# Ruta al archivo fuente inicial 
DATA_PATH = "../data/processed/spotify_clean_modeling.csv"

# Verificar existencia
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"No se encontró el archivo en {DATA_PATH}")

# Carga el archivo CSV
df = pd.read_csv(DATA_PATH)
print(f"Dataset se ha cargado correctamente en un arreglo: {df.shape}")
#display(df.columns.T)
#display(df.head())


Dataset se ha cargado correctamente en un arreglo: (232724, 15)


### 2. Separación de variables predictoras (X) y objetivo (y)

In [75]:
# Separación de variables predictoras (X) y objetivo (y)
X = df.drop(columns=["is_hit","popularity"])
y = df["is_hit"]

display("X shape:", X.shape)
display("X columns:", X.columns.T)
print("Distribución de y:")
display(y.value_counts(normalize=True))


'X shape:'

(232724, 13)

'X columns:'

Index(['genre', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'mode', 'speechiness',
       'tempo', 'time_signature', 'valence'],
      dtype='object')

Distribución de y:


is_hit
0    0.954693
1    0.045307
Name: proportion, dtype: float64

### 3. Código: preprocesamiento (OneHot + ColumnTransformer)

In [76]:
# Preservacion de DataFrames
set_config(transform_output="pandas")

# Identificación de columnas numéricas y categóricas
numeric_cols = X.select_dtypes(include=["float64", "int64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Columnas numéricas:", numeric_cols)
print("Columnas categóricas:", categorical_cols)

# Definición del preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)


Columnas numéricas: ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
Columnas categóricas: ['genre']


### 4. Código: modelo LightGBM + Pipeline

In [77]:
# Definición del modelo LightGBM
model = LGBMClassifier(
    n_estimators=1200,
    learning_rate=0.02,
    num_leaves=128,
    max_depth=-1,
    min_child_samples=20,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.1,
    reg_lambda=0.1,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1,
    verbose=-1
)




# Construcción del pipeline completo
pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", model),
    ]
)


### 5. Código: train_test_split

70% → train (para entrenar modelos)

10% → validation (para buscar el mejor threshold)

20% → test (solo para evaluar al final)

In [78]:
# 1. Split train/test (80 / 20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 2. Split interno train/val (70 / 10)
X_trainModel, X_val, y_trainModel, y_val = train_test_split(
    X_train, y_train,
    test_size=0.125,        # 0.125 de 80% = 10%
    random_state=42,
    stratify=y_train
)

print(X_trainModel.shape, "→ train (70%)")
print(X_val.shape,        "→ validation (10%)")
print(X_test.shape,       "→ test (20%)")

# 3. Restaurar nombres de columnas 
X_trainModel = pd.DataFrame(X_trainModel, columns=X.columns)
X_val        = pd.DataFrame(X_val,        columns=X.columns)
X_test       = pd.DataFrame(X_test,       columns=X.columns)


(162906, 13) → train (70%)
(23273, 13) → validation (10%)
(46545, 13) → test (20%)


### 6. Entrenamiento del modelo LightGBM

In [79]:
# Entrenamiento del modelo
pipeline.fit(X_trainModel, y_trainModel)
print("Entrenamiento completado.")


Entrenamiento completado.


### 7. Código: evaluación inicial (threshold = 0.5)

In [80]:
# Evaluación inicial (umbral por defecto 0.5)
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy:   {acc:.4f}")
print(f"F1-score:   {f1:.4f}")
print(f"ROC-AUC:    {auc:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy:   0.9337
F1-score:   0.5313
ROC-AUC:    0.9603

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.94      0.96     44436
           1       0.39      0.83      0.53      2109

    accuracy                           0.93     46545
   macro avg       0.69      0.88      0.75     46545
weighted avg       0.96      0.93      0.94     46545



### 8. Optimización del umbral de decisión (threshold tuning - F1)


In [81]:
# 1. Probabilidades en VALIDATION
y_proba_val = pipeline.predict_proba(X_val)[:, 1]

# 2. Threshold tuning usando  VALIDATION
thresholds = np.linspace(0.05, 0.95, 30)
f1_scores = []

for t in thresholds:
    preds = (y_proba_val >= t).astype(int)
    f1_scores.append(f1_score(y_val, preds))   

best_t = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

print(f"Mejor threshold (val): {best_t:.3f}")
print(f"Mejor F1-score (val): {best_f1:.4f}")


Mejor threshold (val): 0.764
Mejor F1-score (val): 0.6807


### 9. Evaluación final con threshold optimizado

In [82]:
# 1. Probabilidades en el conjunto de TEST
y_proba_test = pipeline.predict_proba(X_test)[:, 1]

# 2. Aplicar threshold óptimo encontrado en VALIDATION
y_pred_opt = (y_proba_test >= best_t).astype(int)

# 3. Calcular métricas sobre TEST
acc_opt = accuracy_score(y_test, y_pred_opt)
f1_opt  = f1_score(y_test, y_pred_opt)
auc_opt = roc_auc_score(y_test, y_proba_test)

print("=== Métricas con threshold optimizado (TEST) ===")
print(f"Accuracy: {acc_opt:.4f}")
print(f"F1-score:{f1_opt:.4f}")
print(f"ROC-AUC: {auc_opt:.4f}")


=== Métricas con threshold optimizado (TEST) ===
Accuracy: 0.9732
F1-score:0.6853
ROC-AUC: 0.9603


### 10. Guardado del modelo entrenado

In [84]:
# Guardado del modelo entrenado
MODEL_PATH = "../models/spotify_lightgbm_hit_classifier.joblib"
joblib.dump(pipeline, MODEL_PATH)
print(f"Modelo guardado en: {MODEL_PATH}")


Modelo guardado en: ../models/spotify_lightgbm_hit_classifier.joblib
